In [70]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from IPython.display import display
import openslide as os

In [156]:
PROBAS_FP = '../reports/evaluation/d3615ddd-8736-480d-98cc-e4fb3b9af20c/probas/2019_1502-09-1'
WSI_FP = '/mnt/data/scans/AI scans/Prostata/2019_1502-09-1.mrxs'
XML_FP = '/mnt/data/scans/AI scans/Prostata/2019_1502-09-1.xml'

In [157]:
from openslide import open_slide
import numpy as np
import xml.etree.ElementTree as ET

In [144]:
del im

In [175]:
SCALE_FACTOR = 23
slide = open_slide(WSI_FP)

# Dimensions
large_w, large_h = slide.dimensions
small_w = np.floor(large_w / SCALE_FACTOR).astype(int)
small_h = np.floor(large_h / SCALE_FACTOR).astype(int)
print('Resize: {}x{} -> {}x{}'.format(large_w, large_h, small_w, small_h))
level = slide.get_best_level_for_downsample(23)
whole_slide_image = slide.read_region((0,0), level, slide.level_dimensions[level])

# Colors
whole_slide_image = whole_slide_image.convert("RGBA")
whole_slide_image = whole_slide_image.resize((small_w, small_h))

new_image = Image.new('RGBA', whole_slide_image.size, "WHITE")
new_image.paste(whole_slide_image, (0, 0), whole_slide_image)
del whole_slide_image

rectangle_image = Image.new('RGBA', new_image.size, (0,0,0,0))
draw = ImageDraw.Draw(rectangle_image)

# Draw rectangles
with open(PROBAS_FP, 'r') as f_in:
    for idx, line in enumerate(f_in):
        if idx == 0:
            print(line.strip().split(';'))
        else:
            _, row, col, x, y, proba = line.strip().split(';')
            proba = float(proba)
            x = int(x) // 23
            x_size = 299 // 23
            y = int(y) // 23
            y_size = 299 // 23
            R = int(255 * proba)
            G = int(255 * (1 - proba))
            B = int(0)
            A = int(140)
            draw.rectangle([(x, y), (x+x_size, y+y_size)], fill=(R,G,B,A))
            
im = Image.alpha_composite(new_image, rectangle_image)
del rectangle_image
del new_image

# Polygon
polygons = read_polygons(XML_FP, SCALE_FACTOR)
polygon_image = Image.new('RGBA', im.size, (0,0,0,0))
draw = ImageDraw.Draw(polygon_image)
for polygon in polygons:
    polygon.append(polygon[0])
    draw.line(polygon, fill=(255,0,0,255), width=10, joint='curve')
im = Image.alpha_composite(im, polygon_image)

    
# Save
im.save('/home/matejg/demo.png', 'PNG')
print('DONE')

Reize: 105185x221772 -> 4573x9642
['slide_id', 'row', 'col', 'x', 'y', 'tile_proba']
DONE


In [161]:
def read_polygons(annotation_filepath, SCALE_FACTOR):
    tumor_keywords = ['Metastasis', 'Carcinoma', 'Tumor']
    # Read cancer polygon area
    polygons = []
    root = ET.parse(annotation_filepath).getroot()
    for anno_tag in root.findall('Annotations/Annotation'):
        polygon = []
        
        if anno_tag.get('PartOfGroup') not in tumor_keywords:
            continue

        for coord in anno_tag.findall('Coordinates/Coordinate'):
            polygon.append((float(coord.get('X')) / SCALE_FACTOR, float(coord.get('Y')) / SCALE_FACTOR))
        polygons.append(polygon)
    return polygons

In [162]:
read_polygons(XML_FP, 23)

[[(1476.339843478261, 2697.074047826087),
  (1466.7083913043477, 2706.458560869565),
  (1436.5789739130437, 2706.952447826087),
  (1418.7977260869563, 2713.867356521739),
  (1408.4252739130434, 2725.9685782608694),
  (1404.2269869565218, 2735.353091304348),
  (1399.781591304348, 2744.490660869565),
  (1395.830247826087, 2753.6282260869566),
  (1402.4982173913043, 2767.7051652173914),
  (1408.6723, 2774.126186956522),
  (1433.1216043478262, 2770.421704347826),
  (1442.0122304347826, 2805.2433782608696),
  (1455.1012217391303, 2826.7291086956525),
  (1484.4896391304349, 2829.198708695652),
  (1514.1251695652174, 2829.9395391304347),
  (1542.5258173913044, 2832.4093086956523),
  (1573.6430043478263, 2833.150134782609),
  (1601.3028173913044, 2829.445652173913),
  (1631.432234782609, 2834.3848521739133),
  (1670.452104347826, 2832.9031913043477),
  (1718.1158304347828, 2828.9517652173913),
  (1759.6054695652172, 2829.445652173913),
  (1794.6740826086957, 2833.3970782608694),
  (1822.580843